In [6]:
!pip install openai
!pip install torch

In [8]:
!pip install -q requests torch transformers

In [12]:
!pip install python-dotenv


In [ ]:
import os
import requests
from IPython.display import Audio, display, Markdown
from tkinter import Tk, filedialog
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Function to transcribe audio using your backend
def transcribe_audio(audio_file_path):
    url = "http://localhost:5000/api/whisper/transcribe"
    
    with open(audio_file_path, 'rb') as file:
        files = {'audio': file}
        response = requests.post(url, files=files)
        
    if response.status_code == 200:
        print("✅ Audio transcribed successfully!")
        return response.json()['text']
    else:
        print(f"❌ Failed to transcribe audio: {response.status_code}")
        print(response.text)
        return None

# Open file dialog to select audio file
root = Tk()
root.withdraw()  # hide tkinter root window
audio_filename = filedialog.askopenfilename(title="Select an audio file")

if audio_filename:
    print(f"Selected file: {audio_filename}")
    transcript = transcribe_audio(audio_filename)
    if transcript:
        print("Transcript:")
        print(transcript)
        display(Markdown(transcript))
        
        # Optional: Save to history
        user_id = input("Enter your user ID to save to history (or press Enter to skip): ")
        if user_id:
            history_url = "http://localhost:5000/api/history/save"
            data = {
                "userId": user_id,
                "original_audio": os.path.basename(audio_filename),
                "transcript": transcript
            }
            save_response = requests.post(history_url, json=data)
            if save_response.status_code == 200:
                print("✅ Saved to history!")
            else:
                print(f"❌ Failed to save to history: {save_response.text}")
else:
    print("No file selected")